In [7]:
import socket

listen_socket = socket.socket()
listen_socket.bind(('127.0.0.1', 12345))
listen_socket.listen()

game_socket, addr = listen_socket.accept()

# server input word first, then display 
word = input("Hangman word: ")
left = len(word)

game_socket.sendall(b'START,' + str(len(word)).encode() + b'\n')

while True: 
    rx_msg_str = ''
    while '\n' not in rx_msg_str:
        rx_msg_str += game_socket.recv(1024).decode()
    
    if rx_msg_str[:5] == "GUESS": 
        guess_letter = rx_msg_str[6:-1]
        guess_index = ''
        for i in range(len(word)):
            if word[i] == guess_letter:
                guess_index += str(i) + ','
                left -= 1
        guess_index = guess_index[:-1]
        if left == 0: 
            game_socket.sendall(b'WIN\n')
        if guess_index == '':
            game_socket.sendall(b'GUESS\n')
        else:
            game_socket.sendall(b'GUESS,' + guess_index.encode() + b'\n')
        
    if rx_msg_str[:5] == "HWORD":
        guess_word = rx_msg_str[6:-1]
        if guess_word == word:
            game_socket.sendall(b'WIN\n')
            break
        else:
            game_socket.sendall(b'LOSE\n')
            break
            
    if rx_msg_str[:4] == "QUIT": 
        game_socket.sendall(b'QUIT\n')
        break 

game_socket.close()   
listen_socket.close()   

Hangman word: hello


ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host